<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/answer_Generation_jais_family_590m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `week1 test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `week1 t

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import json
import os
from google.colab import files
from IPython.display import display, FileLink

In [ ]:

model_path = "inceptionai/jais-family-590m"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_auth_token=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)



/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/247 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/inceptionai/jais-family-590m:
- configuration_jais.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_jais.py:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/inceptionai/jais-family-590m:
- modeling_jais.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json:   0%|          | 0.00/20.2k [00:00<?, ?B/s]

model-00001-of-00001.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

In [ ]:
def get_response(text, temperature, do_sample, top_p, num_return_sequences):
    encoding = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=100)
    input_ids = encoding.input_ids.to(model.device)  # Ensure input is sent to the model's device
    attention_mask = encoding.attention_mask.to(model.device)

    generate_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=100,
        num_beams=10,
        top_p=top_p,
        temperature=temperature,
        repetition_penalty=1.2,
        do_sample=do_sample,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode all generated sequences
    return [tokenizer.decode(ids, skip_special_tokens=True) for ids in generate_ids]


In [ ]:
def generate_answer(question, num_samples):
    prompt = f" أجب على السؤال التالي بجملة واحدة موجزة ولكن كاملة باللغة العربية.\nQuestion: {question}\nAnswer:"

    # Generate the first 9 answers with temperature=1.0
    outputs_high_temp = get_response(
        prompt,
        temperature=1.0,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=num_samples - 1
    )

    # Generate the last answer with low temperature
    outputs_low_temp = get_response(
        prompt,
        temperature=0.1,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=1
    )

    # Combine the results
    responses = [
        output.split("Answer:")[-1].strip() for output in outputs_high_temp
    ] + [
        outputs_low_temp[0].split("Answer:")[-1].strip()
    ]

    return responses

In [ ]:
# Main function
def main():
    # Step 1: Upload and read the file
    file_path = '/content/train-open.json'
    if os.path.exists(file_path):
        print("File found")
    else:
        print("File not found")
        return

    with open(file_path, 'r', encoding='utf-8') as file:
         data = json.load(file)

    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Initialize an empty list to store the results
    results = []

    # Step 2:  Generate answers for a sample of questions
    for index, row in df.sample(n=50).iterrows() :
        question = row['question']
        original_answer = row.get('answer', "")  # Use empty string if no original answer provided
        question_id = row.get('question_id', index)  # Use index if no question_id provided

        # Generate 10 answers for the 50 questions
        generated_answers = generate_answer(question=question, num_samples=10)

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers
        })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)

    filename = "generated_answers.json"
    # Step 3: Save results to a JSON file
    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    display(FileLink(filename))
    files.download("generated_answersjais.json")

    #what i think next stpes is
    # Step 4: Clustering
    # Step 5: Calculate probability
    # Step 6: Calculate semantic entropy
    # Step 7: show results

# Run the main function
if __name__ == "__main__":
    main()

File found
   question_id                                           question  \
0      1102659       ما هي أهداف ومهام جهاز الأمن الوطني العراقي؟   
1      1173725  مع أي نوادي لعب إنريكي جارا ساغوير خلال مسيرته...   
2       952582         ما هي الأنساب الكاملة للملك محمد الخامس؟\n   
3      1181660                         من هو مبتكر  شخصية غامورا؟   
4      1192692                            متي صدر فيلم وقت الغجر؟   
5      1092943   متى تلقى لوك دي كلابير إشعارًا باسمه لأول مرة؟\n   
6      1199718                            اين وُلد فوديبا كيتا؟\n   
7      1038367  ما هي القدرات الكبيرة التي أظهرها روبرت شومان ...   
8      1002364             من المشاركين في توقيع اتفاقية أوسلو؟\n   
9       701306               كيف ساهم حزب الله في الحرب البوسنية؟   
10      674993  ما هو اليوم الذي يوافق اليوم السابع والعشرون ب...   
11     1184904                ما هي الصيغة الكيميائية لكولبكيت؟\n   
12     1124806     ما هو تقييم مؤرخي الفن لجوتشو وأعماله؟\n\n\n\n   
13     1117537         

/content/generated_answers.json

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>